# Space Station 🚀
```
           ___
     |     | |
    / \    | |
   |--o|===|-|
   |---|   |N|
  /     \  |A|
 | Snow  | |S|
 | Ex    |=|A|
 |       | | |
 |_______| |_|
  |@| |@|  | |
  *** *** 
 ***********
*************
```



## SnowEx Hackweek 2022

### "Hack"-tronauts
- **Team Leader:** Micah "Sub-naught" Johnson
- Beau Uriona
- Wyatt Reis
- Angela Rienzo
- Danny Hogan
- Jordan Beer
- Ibrahim Olalekan Alabi
- Dylan Craaybeek
- Micah "Prime" Sandusky 
- Ross Palomaki

## Motivation

1. Can we build a pipeline to connect a SNOTEL station's observations to the closest relavent SnowEx data of interest?
2. How does a SNOTEL station represent surrounding measured snow from ASO's snow depth product?
3. What are the main drivers for the surrounding heterogeneity (distance, predominant wind, slope, veg cover, aspect, elevation, etc.)?

## Process
![Diagram showing raster to depth](./figures/raster-depth-diff.jpg)

## Locations

### Map with All Locations
![Show ArcPro figures here](figures/ZoomedOut.png)

### Location close-up with Buffer
![Show ArcPro figures here](figures/ZoomedIn.png)

## Results and Visualizations

### Visualize depth difference between snow depth raster and SNOTEL snow dpeth

In [2]:
import geopandas as gpd
import pandas as pd
import folium
import numpy as np

In [3]:
import pickle
with open('../one_obj_to_rule_them_all.pkl', 'rb') as p:
    obj = pickle.load(p)

data = obj['data']
meta_snowex = obj['meta']['snowex']
meta_awdb = obj['meta']['awdb']
data['difference_swe'] = data['swe_awdb'] - data['swe_snowex']
data['difference_depth'] = data['depth_awdb'] - data['depth_snowex']
data.index = data.index.astype(str)
data['date'] = data['date'].astype(str)
data.head()

,snowex_pit_id,instrument,snowex_id,geometry,swe_snowex,depth_snowex,date,swe_awdb,depth_awdb,awdb_id,distance,difference_swe,difference_depth
2019-12-20,CASHFO_20191220_1231,manual,Forest,POINT (-120.24211 39.42968),91.0,250.0,2019-12-20,163.0,508.0,539:CA:SNTL,5159.0,72.0,258.0
2019-12-20,CASHOP_20191220_1123,manual,Open,POINT (-120.23981 39.43041),135.0,410.0,2019-12-20,163.0,508.0,539:CA:SNTL,5294.0,28.0,98.0
2020-01-29,CASHFO_20200129_1053,manual,Forest,POINT (-120.24203 39.42957),103.0,300.0,2020-01-29,251.0,838.0,539:CA:SNTL,5172.0,148.0,538.0
2020-01-29,CASHOP_20200129_1057,manual,Open,POINT (-120.23982 39.43036),194.0,500.0,2020-01-29,251.0,838.0,539:CA:SNTL,5296.0,57.0,338.0
2020-01-29,CASHT4_20200129_1415,manual,Tower 4,POINT (-120.29898 39.42216),429.0,1200.0,2020-01-29,251.0,838.0,539:CA:SNTL,3446.0,-178.0,-362.0


In [8]:
# Show Point map with difference values 
cmap_col = 'difference_depth'
#cmap_col = 'distance'

# Create map object
m = folium.Map(location=[40, -113], zoom_start=5, tiles=None, 
                     control_scale=True)
folium.TileLayer('Stamen terrain', name='Terrain basemap', control=True).add_to(m)
folium.TileLayer('CartoDB positron', name='Light basemap', control=True).add_to(m)


# Colormap for points
from branca.colormap import linear as linear_cm
if 'difference' in cmap_col:
    # Divergent colormap
    cmap = linear_cm.RdBu_09.scale(min(data[cmap_col]), max(data[cmap_col]))
else:
    # Sequential colormap
    cmap = linear_cm.YlGnBu_09.scale(min(data[cmap_col]), max(data[cmap_col]))
    
viz_colors = [cmap(i) if not pd.isnull(i) else 'black' for i in data.loc[data['instrument']=='manual', cmap_col]]


# Clean up columns for tooltip
old_cols = data.columns
new_cols = ['Pit ID','Instrument','SnowEx site','geometry','SWE (pit, mm)','Snow depth (pit, mm)',
            'Date collected','SWE (AWDB station, mm)','Snow depth (AWDB station, mm)',
            'AWDB Station ID','Distance from pit to station (m)', 'SWE difference (mm)',
            'Snow depth difference (mm)']
data.columns = new_cols
tooltip_cols = ['Date collected','SnowEx site','Pit ID','Snow depth (pit, mm)','SWE (pit, mm)',
                'AWDB Station ID','Snow depth (AWDB station, mm)','SWE (AWDB station, mm)',
                'Snow depth difference (mm)','SWE difference (mm)','Distance from pit to station (m)']

# Explore layers
awdb_layer = meta_awdb.explore(
    name='AWDB stations', 
    color='red',
    m=m,
    tooltip=['site_name', 'site_id'],
    marker_kwds={'radius': 10},
    style_kwds={'color': '#4a4a4a','weight': 1}
)

snowex_layer = meta_snowex.explore(
    name='All SnowEx pits', 
    color='orange',
    m=m,
    tooltip=['site_name', 'site_id'],
    marker_kwds={'radius': 10},
    style_kwds={'color': '#4a4a4a','weight': 1}
)

pit_layer = data[data['Instrument']=='manual'].explore(
    name='Pit data',
    color=viz_colors,
    m=m,
    tooltip=tooltip_cols,
    marker_kwds={'radius': 10},
    style_kwds={'color': '#4a4a4a','weight': 1}
)

data.columns = old_cols

# # Lidar layer
# import rasterio
# with rasterio.open('gm_2022_02_02_depth.tif') as src:
#     lidar = src.read()
#     lidar_bounds = src.bounds
#     lidar_crs = src.crs
    
# lidar_layer = folium.raster_layers.ImageOverlay(
#     image=lidar[0],
#     bounds=lidar_bounds,
#     name='Lidar data'
# ).add_to(m)


m.add_child(folium.LayerControl())
cmap.caption = cmap_col
cmap.add_to(m)
m


### Some rudimentary stats 

![Show ArcPro figures here](figures/histogram.png)
![Show ArcPro figures here](figures/line_plot.png)

## Next Steps

- Analyze with respect to other terrain variables
- Look at other SnowEx flights and products that align with SNOTEL sites
- Snow poles as "stations"
- Expand to look at any snow depth lidar products
- Comparing to snow model output
